In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import json, requests, timeit
from mlxtend.frequent_patterns import apriori, fpgrowth
import pandas as pd


***Implement Apriori Algorithm***<br>
The Apriori algorithm is used to extract frequent itemsets from a dataset using the concept of support. Below is the approach:

Key Steps of Apriori:

1. Generate candidate itemsets of length 1 (C1) and count their frequency.
2. Prune itemsets that don’t meet the minimum support threshold.
3. Generate candidate itemsets of length k+1 by combining frequent itemsets of length k (Ck+1).
4. Repeat until no more itemsets can be generated.


In [47]:
def implemented_apriori(dataset, min_support):
    # Step 1: Count frequency of each item
    item_counts = defaultdict(int)
    for transaction in dataset:
        for item in transaction:
            item_counts[item] += 1

    # Step 2: Filter items by min_support
    num_transactions = len(dataset)
    min_count = num_transactions * min_support
    frequent_items = {item for item, count in item_counts.items() if count >= min_count}

    # Step 3: Generate frequent itemsets
    frequent_itemsets = []
    k = 1
    current_itemsets = [{item} for item in frequent_items]

    while current_itemsets:
        # Count frequency of current itemsets
        itemset_counts = defaultdict(int)
        for transaction in dataset:
            for itemset in current_itemsets:
                if itemset.issubset(transaction):
                    itemset_counts[frozenset(itemset)] += 1

        # Filter by min_support
        current_itemsets = [
            itemset for itemset, count in itemset_counts.items()
            if count >= min_count
        ]
        frequent_itemsets.extend(current_itemsets)

        # Generate combinations for next level
        k += 1
        current_itemsets = [set(a).union(set(b)) for a in current_itemsets for b in current_itemsets if len(a.union(b)) == k]

    return frequent_itemsets


***2. Load and Transform COCO Dataset***

The COCO dataset contains image annotations. Each image is considered a transaction, and the annotations are the items.


In [48]:
url = "https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/modified_coco.json"

file_path = "modified_coco.json"

# Download the file
response = requests.get(url)


with open(file_path, "wb") as file:
    file.write(response.content)
    print(f"File downloaded successfully: {file_path}")




File downloaded successfully: modified_coco.json


In [49]:
with open(file_path) as file:
    data = json.load(file)

dataset = [image['annotations'] for image in data]
dataset[:5]

[['car', 'car', 'train', 'stop sign'],
 ['person',
  'person',
  'person',
  'person',
  'person',
  'bench',
  'chair',
  'chair',
  'chair',
  'chair',
  'chair',
  'chair',
  'potted plant',
  'potted plant',
  'potted plant',
  'dining table',
  'dining table',
  'dining table'],
 ['stop sign'],
 ['person', 'fire hydrant'],
 ['bicycle', 'bicycle', 'fire hydrant']]

***3. Run Apriori on COCO Dataset***

In [50]:
# Run Apriori with a minimum support of 0.02
frequent_itemsets = implemented_apriori(dataset, min_support=0.02)

# Display the frequent itemsets
print("Frequent Itemsets with min_support=0.02:")
for itemset in frequent_itemsets:
    print(itemset)


Frequent Itemsets with min_support=0.02:
frozenset({'train'})
frozenset({'stop sign'})
frozenset({'car'})
frozenset({'bench'})
frozenset({'person'})
frozenset({'chair'})
frozenset({'potted plant'})
frozenset({'dining table'})
frozenset({'fire hydrant'})
frozenset({'bicycle'})
frozenset({'bus'})
frozenset({'backpack'})
frozenset({'parking meter'})
frozenset({'truck'})
frozenset({'umbrella'})
frozenset({'traffic light'})
frozenset({'clock'})
frozenset({'motorcycle'})
frozenset({'dog'})
frozenset({'handbag'})
frozenset({'baseball bat'})
frozenset({'sports ball'})
frozenset({'tennis racket'})
frozenset({'bird'})
frozenset({'baseball glove'})
frozenset({'cell phone'})
frozenset({'bottle'})
frozenset({'suitcase'})
frozenset({'skateboard'})
frozenset({'cup'})
frozenset({'stop sign', 'car'})
frozenset({'person', 'bench'})
frozenset({'bench', 'chair'})
frozenset({'bench', 'potted plant'})
frozenset({'dining table', 'bench'})
frozenset({'person', 'chair'})
frozenset({'person', 'potted plant'})
f

***4. Compare Results with Mlxtend Library***

To verify the results, convert the dataset into a binary matrix and use Mlxtend's apriori() and fpgrowth() functions.

In [51]:
# Extract all unique items
all_items = list({item for transaction in dataset for item in transaction})

# Create binary matrix
matrix = []
for transaction in dataset:
    matrix.append([1 if item in transaction else 0 for item in all_items])

df_matrix = pd.DataFrame(matrix, columns=all_items)


In [52]:
# Use Mlxtend Apriori
mlxtend_apriori = apriori(df_matrix, min_support=0.02, use_colnames=True)
print("\nMlxtend Apriori Results:")
print(mlxtend_apriori)



Mlxtend Apriori Results:
     support                                       itemsets
0     0.0368                                  (sports ball)
1     0.1346                                 (fire hydrant)
2     0.0276                                          (dog)
3     0.0474                                     (umbrella)
4     0.0354                                   (cell phone)
..       ...                                            ...
139   0.0246          (bicycle, traffic light, person, car)
140   0.0224         (traffic light, person, car, backpack)
141   0.0366          (traffic light, person, car, handbag)
142   0.0438              (traffic light, person, car, bus)
143   0.0238  (baseball glove, bench, baseball bat, person)

[144 rows x 2 columns]


c:\Users\taha_kms\AppData\Local\Programs\Python\Python39\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [53]:

# Use Mlxtend FP-Growth
mlxtend_fpgrowth = fpgrowth(df_matrix, min_support=0.02,)
print("\nMlxtend FP-Growth Results:")
print(mlxtend_fpgrowth)


c:\Users\taha_kms\AppData\Local\Programs\Python\Python39\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(



Mlxtend FP-Growth Results:
     support      itemsets
0     0.3704          (22)
1     0.1332          (14)
2     0.0492          (72)
3     0.5886          (50)
4     0.4338          (65)
..       ...           ...
139   0.0226      (50, 75)
140   0.0202  (65, 50, 75)
141   0.0342      (41, 50)
142   0.0254      (65, 41)
143   0.0254  (65, 41, 50)

[144 rows x 2 columns]


***5. Compare Execution Times***

In [55]:
# Apriori time
apriori_time = timeit.timeit(lambda: implemented_apriori(dataset, min_support=0.02), number=1)
print(f"\nExecution time of custom Apriori: {apriori_time:.4f} seconds")



Execution time of custom Apriori: 8.9613 seconds


In [56]:
# Mlxtend Apriori time
mlxtend_apriori_time = timeit.timeit(lambda: apriori(df_matrix, min_support=0.02, use_colnames=True), number=1)
print(f"Execution time of Mlxtend Apriori: {mlxtend_apriori_time:.4f} seconds")


Execution time of Mlxtend Apriori: 0.0606 seconds


c:\Users\taha_kms\AppData\Local\Programs\Python\Python39\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [57]:
# Mlxtend FP-Growth time
mlxtend_fpgrowth_time = timeit.timeit(lambda: fpgrowth(df_matrix, min_support=0.02, use_colnames=True), number=1)
print(f"Execution time of Mlxtend FP-Growth: {mlxtend_fpgrowth_time:.4f} seconds")


c:\Users\taha_kms\AppData\Local\Programs\Python\Python39\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Execution time of Mlxtend FP-Growth: 1.2898 seconds
